
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [1]:
import threading
import time
import random

class Barberia:
    def __init__(self):
        # Mecanismos y variables de control clientes y acceso a la sala
        ####### inicializa cada uno con el tipo y valor oportuno####
        #####puedes cambiar los nombres o añadir nuevos#######

        self.MAX_CLIENTES = 3
        # Controla la entrada en la sala de espera
        self.entrarEnSala = threading.Semaphore(self.MAX_CLIENTES) ## Creo que es un semáforo porque a la sala pueden entrar 3 personas.
        # Controla el número de clientes en la sala
        self.clientesEnSala = 0
        # Garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala = threading.Lock() ## No estoy seguro si sería un semáforo o mejor un Lock.
        
        # Mecanismos y variables de contrrol del barbero
        ####### inicializa cada uno con el tipo y valor oportuno #######
        #####puedes cambiar los nombres o añadir nuevos#######
        # Controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Lock() ## ¿Creo que sería un Lock porque el sillón es excluyente...? ¿Sería RLock si es el hilo del barbero el que hace acquiere para cada cliente de manera recursiva. ¿O condition??
        # Controla lel estado del barbero
        self.barberoOcupado = False ## Creo que es un cerrojo¿?, para usarlo luego con un acquire y un release
        # Puede poner al barbero a dormir y ser despertado
        self.dormir = threading.Condition() ## No estoy seguro de esto. With y wait...() y notify...
        

    def actividadBarbero(self):
        while True:
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###

            ### consultar en exclusion mutua los clientes que hay en la sala ###
            
            # Cuando se despierta el barbero, mira lo primero si tiene que dormir. Si hay gente, quita uno de la sala y se pone en estado ocupad
            with self.dormir:
                ### si no hay clientes esperando se pone a dormir hasta que le despierten ####
                if self.clientesEnSala == 0:
                    print("El barbero se va a dormir. \n")
                    self.dormir.wait()

                print("El barbero da paso a un cliente\n")
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
                with self.exclusionMutuaSala:
                    self.clientesEnSala -= 1 # Ojo, en exclusión mutua?
                
                self.barberoOcupado = True
            
                print("El barbero corta el pelo a un cliente. \n")
                time.sleep(5)
                
            ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO Y AVISAR A LOS CLIENTES DE LA SALA ###
            with self.exclusionMutuaSala:
                self.barberoOcupado = False
                print("El barbero ha terminado de cortar el pelo\n")  
            
            
    def actividadCliente(self):
            print("El cliente " + threading.current_thread().name + " llega a la puerta de la barbería \n")
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL ###
            ### según estado entrar en la sala o no ####
            with self.exclusionMutuaSala:
                if self.clientesEnSala < self.MAX_CLIENTES:
                    self.clientesEnSala += 1
                    ### según estado, despertar al babero o no ####
                    #if (self.clientesEnSala == 1):
                    with self.dormir:
                        self.dormir.notify()
                else:
                    print(" La sala de espera está llena. El cliente " + threading.current_thread().name  + " se va.")
                    # Habría que destruir este hilo o hacer algo... de lo contrario, el cliente sigue cogiendo el sillón y se pela

            ### consultar en exclusion mutua el estado del barbero ###
            self.sillon.acquire()    
            print ("Esta en el sillón el " + threading.current_thread().name  +".\n")
            time.sleep(3)
            print ("El "+threading.current_thread().name+" abandona la barbería PELADO. \n")
            self.sillon.release()


if __name__=="__main__":
    barberia= Barberia()
        
    ### INCLUYE EL CÓDIGO NECESARIO PARA CREAR E INICIALIZAR LOS HILOS  ###
       
    # Crear e iniciar el hilo del barbero
    barbero = threading.Thread(target = barberia.actividadBarbero)
    barbero.start()
        
    ### clientes####
    # Crear e iniciar hilos para varios clientes
    for i in range(10):
        cliente = threading.Thread(target = barberia.actividadCliente, name=f'Cliente {i}')
        time.sleep(random.uniform(0.1, 0.5))  # Simula la llegada de clientes en momentos aleatorios
        cliente.start()


El barbero se va a dormir. 

El cliente Cliente 0 llega a la puerta de la barbería 

Esta en el sillón el Cliente 0.

El barbero da paso a un cliente

El barbero corta el pelo a un cliente. 

El cliente Cliente 1 llega a la puerta de la barbería 

El cliente Cliente 2 llega a la puerta de la barbería 

El cliente Cliente 3 llega a la puerta de la barbería 

El cliente Cliente 4 llega a la puerta de la barbería 

El cliente Cliente 5 llega a la puerta de la barbería 

El cliente Cliente 6 llega a la puerta de la barbería 

El cliente Cliente 7 llega a la puerta de la barbería 

El cliente Cliente 8 llega a la puerta de la barbería 

El cliente Cliente 9 llega a la puerta de la barbería 



El Cliente 0 abandona la barbería PELADO. 

Esta en el sillón el Cliente 1.
 La sala de espera está llena. El cliente Cliente 4 se va.
 La sala de espera está llena. El cliente Cliente 5 se va.
 La sala de espera está llena. El cliente Cliente 6 se va.
 La sala de espera está llena. El cliente Cliente 7 se va.
 La sala de espera está llena. El cliente Cliente 8 se va.
 La sala de espera está llena. El cliente Cliente 9 se va.
El barbero ha terminado de cortar el pelo

El barbero da paso a un cliente

El barbero corta el pelo a un cliente. 


El Cliente 1 abandona la barbería PELADO. 

Esta en el sillón el Cliente 2.

El barbero ha terminado de cortar el pelo

El barbero da paso a un cliente

El barbero corta el pelo a un cliente. 

El Cliente 2 abandona la barbería PELADO. 

Esta en el sillón el Cliente 3.

El Cliente 3 abandona la barbería PELADO. 

Esta en el sillón el Cliente 4.

El barbero ha terminado de cortar el pelo

El barbero da paso a un cliente

El barbero corta el pelo a u